In [21]:
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor

__file__ = Path('submissions') /  'external_data' /  'estimator.py'

def _encode(X):
    #cyclical encoding of dates
    X = X.copy()
    year_norm = 2 * math.pi * X['date'].dt.year / X['date'].dt.year.max()
    month_norm = 2 * math.pi * X['date'].dt.month / X['date'].dt.month.max()
    day_norm = 2 * math.pi * X['date'].dt.day / X['date'].dt.day.max()
    weekday_norm = 2 * math.pi * X['date'].dt.weekday / X['date'].dt.weekday.max()
    hour_norm = 2 * math.pi * X['date'].dt.hour / X['date'].dt.hour.max()
    X.loc[:, 'year_sin'] = np.sin(year_norm)
    X.loc[:, 'year_cos'] = np.cos(year_norm)
    X.loc[:, 'month_sin'] = np.sin(month_norm)
    X.loc[:, 'month_cos'] = np.cos(month_norm)
    X.loc[:, 'day_sin'] = np.sin(day_norm)
    X.loc[:, 'day_cos'] = np.cos(day_norm)
    X.loc[:, 'weekday_sin'] = np.sin(weekday_norm)
    X.loc[:, 'weekday_cos'] = np.cos(weekday_norm)
    X.loc[:, 'hour_sin'] = np.sin(hour_norm)
    X.loc[:, 'hour_cos'] = np.cos(hour_norm)
    #encode dates
    X.loc[:, 'year'] = X['date'].dt.year
    X.loc[:, 'month'] = X['date'].dt.month
    X.loc[:, 'day'] = X['date'].dt.day
    X.loc[:, 'weekday'] = X['date'].dt.weekday
    X.loc[:, 'hour'] = X['date'].dt.hour
    return X


def _merge_external_data(X):
    file_path = Path(__file__).parent / 'external_data.csv'
    df_ext = pd.read_csv(file_path, parse_dates=['date'])
    X = X.copy()
    # When using merge_asof left frame need to be sorted
    X['orig_index'] = np.arange(X.shape[0])
    X = pd.merge_asof(X.sort_values('date'), df_ext[['date', 't', 'ff', 'u', 'brent', 'vacances_scol', 'bank_hol']].sort_values('date'), on='date')
    # Sort back to the original order
    X = X.sort_values('orig_index')
    del X['orig_index']
    return X
    # Finally we can drop the original columns from the dataframe



In [22]:
def get_estimator():
    date_encoder = FunctionTransformer(_encode)
    cycl_cols = ['month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos']

    categorical_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    categorical_cols = ["site_name", "counter_name"]
    holiday_cols =  ['bank_hol', 'vacances_scol']
    numeric_cols = ['t', 'ff', 'u', 'brent']

    preprocessor = ColumnTransformer(
        [
            ('date', 'passthrough', cycl_cols),
            ('year', 'passthrough', ['year']),
            ('holiday', 'passthrough', holiday_cols),
            ('cat', categorical_encoder, categorical_cols),
            ('numeric', 'passthrough', numeric_cols)
        ]
    )
    regressor = HistGradientBoostingRegressor(random_state=0)

    pipe = make_pipeline(
        FunctionTransformer(_merge_external_data, validate=False), date_encoder, preprocessor, regressor)

    return pipe


In [3]:
import problem

X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

In [4]:
for param_name in get_estimator().get_params().keys():
    print(param_name)

memory
steps
verbose
functiontransformer-1
functiontransformer-2
columntransformer
histgradientboostingregressor
functiontransformer-1__accept_sparse
functiontransformer-1__check_inverse
functiontransformer-1__func
functiontransformer-1__inv_kw_args
functiontransformer-1__inverse_func
functiontransformer-1__kw_args
functiontransformer-1__validate
functiontransformer-2__accept_sparse
functiontransformer-2__check_inverse
functiontransformer-2__func
functiontransformer-2__inv_kw_args
functiontransformer-2__inverse_func
functiontransformer-2__kw_args
functiontransformer-2__validate
columntransformer__n_jobs
columntransformer__remainder
columntransformer__sparse_threshold
columntransformer__transformer_weights
columntransformer__transformers
columntransformer__verbose
columntransformer__verbose_feature_names_out
columntransformer__date
columntransformer__year
columntransformer__holiday
columntransformer__cat
columntransformer__numeric
columntransformer__cat__categories
columntransformer__ca

In [27]:
from sklearn.model_selection import GridSearchCV
import time

param_grid = {'histgradientboostingregressor__learning_rate': (0.01, 0.1, 1, 10),
             'histgradientboostingregressor__max_leaf_nodes': (3, 10, 30)}

model_grid = GridSearchCV(get_estimator(), param_grid=param_grid, n_jobs=7)
start_time = time.time()
model_grid.fit(X_train, y_train)
print(time.time() - start_time)

print(f"The best set of parameters is: {model_grid.best_params_}")

167.5158176422119
The best set of parameters is: {'histgradientboostingregressor__learning_rate': 0.1, 'histgradientboostingregressor__max_leaf_nodes': 3}


In [29]:
from sklearn.model_selection import cross_val_score

train_scores = cross_val_score(model_grid, X_train, y_train)
test_scores = cross_val_score(model_grid, X_test, y_test)

print(f"The train accuracy score is: {train_scores.mean():.2f} +- {train_scores.std():.2f}")
print(f"The different scores obtained are: \n{train_scores}")

print(f"The test accuracy score is: {test_scores.mean():.2f} +- {test_scores.std():.2f}")
print(f"The different scores obtained are: \n{test_scores}")

print(f"The best set of parameters is: {model_grid.best_params_}")

The train accuracy score is: 0.56 +- 0.11
The different scores obtained are: 
[0.56062665 0.70574582 0.58571728 0.58393854 0.35100386]
The validation accuracy score is: 0.52 +- 0.07
The different scores obtained are: 
[0.55676324 0.63239756 0.4270575  0.50191016 0.49208823]
The best set of parameters is: {'histgradientboostingregressor__learning_rate': 0.1, 'histgradientboostingregressor__max_leaf_nodes': 3}
